In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Load the dataset
max_features = 10000  # vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Pad sequences
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

def create_model(dim=128, learning_rate=0.001):
    """
    Creates and compiles a SimpleRNN model.
    Hyperparameters are passed as arguments.
    """
    model = Sequential()
    model.add(Input(shape=(max_len,)))
    model.add(Embedding(max_features, dim))
    model.add(SimpleRNN(dim, activation="tanh"))
    model.add(Dense(1, activation="sigmoid"))

    # Create the optimizer 
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

# 2. Use KerasClassifier 
model = KerasClassifier(model=create_model, verbose=1)

param_grid = {
    'batch_size': [32, 64],
    'epochs': [3, 5],
    'model__dim': [100, 128],
    'model__learning_rate': [0.001, 0.0005],
}

# Create and run the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2,cv=3)
grid_result = grid.fit(X_train, y_train)

# Summarize the results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Mean: {mean}, Stdev: {stdev} with: {param}")

### Best: 0.8193222569315539 using {'batch_size': 64, 'epochs': 5, 'model__dim': 100, 'model__learning_rate': 0.0005}

# GridSearchCV Training Log

> Fitting 3 folds for each of 16 candidates, totalling 48 fits

-----

## Cross-Validation Folds

### `batch_size=32, epochs=3, model__dim=100, model__learning_rate=0.001`

  * **Fold 1:** Total time: `1.5min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.5726 - loss: 0.6698
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 39s 39ms/step - accuracy: 0.7700 - loss: 0.4889
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8316 - loss: 0.3932
    ```
  * **Fold 2:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5289 - loss: 0.6957
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6969 - loss: 0.5793
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6932 - loss: 0.5867
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - accuracy: 0.5544 - loss: 0.6783
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7317 - loss: 0.5319
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7486 - loss: 0.5151
    ```

-----

### `batch_size=32, epochs=3, model__dim=100, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5561 - loss: 0.6767
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8313 - loss: 0.3892
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8723 - loss: 0.3212
    ```
  * **Fold 2:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.5361 - loss: 0.6896
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6971 - loss: 0.5977
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8443 - loss: 0.3694
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5335 - loss: 0.6884
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 43ms/step - accuracy: 0.7575 - loss: 0.5229
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7520 - loss: 0.5393
    ```

-----

### `batch_size=32, epochs=3, model__dim=128, model__learning_rate=0.001`

  * **Fold 1:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - accuracy: 0.5291 - loss: 0.6896
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7102 - loss: 0.5523
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.7475 - loss: 0.5237
    ```
  * **Fold 2:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.5354 - loss: 0.6898
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.6820 - loss: 0.6059
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7997 - loss: 0.4419
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5248 - loss: 0.6945
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.6616 - loss: 0.6162
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7112 - loss: 0.5682
    ```

-----

### `batch_size=32, epochs=3, model__dim=128, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 41ms/step - accuracy: 0.5455 - loss: 0.6838
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7144 - loss: 0.5525
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7549 - loss: 0.5239
    ```
  * **Fold 2:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5462 - loss: 0.6859
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7700 - loss: 0.4969
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8367 - loss: 0.3713
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5307 - loss: 0.6904
    Epoch 2/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.6718 - loss: 0.6251
    Epoch 3/3
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7985 - loss: 0.4376
    ```

-----

### `batch_size=32, epochs=5, model__dim=100, model__learning_rate=0.001`

  * **Fold 1:** Total time: `1.7min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5431 - loss: 0.6847
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.6549 - loss: 0.6388
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.7233 - loss: 0.5405
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.8108 - loss: 0.4260
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.8108 - loss: 0.4058
    ```
  * **Fold 2:** Total time: `1.7min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - accuracy: 0.5630 - loss: 0.6738
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.7430 - loss: 0.5217
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.8321 - loss: 0.3829
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7664 - loss: 0.4754
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6855 - loss: 0.5831
    ```
  * **Fold 3:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.5455 - loss: 0.6790
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7730 - loss: 0.4762
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7574 - loss: 0.5196
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7249 - loss: 0.5514
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7292 - loss: 0.5207
    ```

-----

### `batch_size=32, epochs=5, model__dim=100, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.5584 - loss: 0.6792
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7655 - loss: 0.5051
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8575 - loss: 0.3359
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8808 - loss: 0.2986
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6991 - loss: 0.5723
    ```
  * **Fold 2:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5492 - loss: 0.6878
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7386 - loss: 0.5530
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7780 - loss: 0.4623
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8590 - loss: 0.3324
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9302 - loss: 0.1882
    ```
  * **Fold 3:** Total time: `1.7min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5332 - loss: 0.6850
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7965 - loss: 0.4705
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9358 - loss: 0.1942
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9878 - loss: 0.0523
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9974 - loss: 0.0161
    ```

-----

### `batch_size=32, epochs=5, model__dim=128, model__learning_rate=0.001`

  * **Fold 1:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5467 - loss: 0.6816
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.6586 - loss: 0.6286
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7282 - loss: 0.5280
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.8128 - loss: 0.4181
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8515 - loss: 0.3497
    ```
  * **Fold 2:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5481 - loss: 0.6849
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7230 - loss: 0.5522
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6989 - loss: 0.5905
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7653 - loss: 0.4935
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8140 - loss: 0.4292
    ```
  * **Fold 3:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.5385 - loss: 0.6850
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.6739 - loss: 0.6095
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.7403 - loss: 0.5141
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7939 - loss: 0.4475
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.8561 - loss: 0.3347
    ```

-----

### `batch_size=32, epochs=5, model__dim=128, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5375 - loss: 0.6841
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7107 - loss: 0.5826
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7855 - loss: 0.4632
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8791 - loss: 0.2977
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8598 - loss: 0.3385
    ```
  * **Fold 2:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5693 - loss: 0.6699
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8304 - loss: 0.3855
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.8626 - loss: 0.3279
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7757 - loss: 0.4803
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8816 - loss: 0.2952
    ```
  * **Fold 3:** Total time: `1.8min`
    ```
    Epoch 1/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 23s 40ms/step - accuracy: 0.5476 - loss: 0.6844
    Epoch 2/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7148 - loss: 0.5694
    Epoch 3/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7993 - loss: 0.4430
    Epoch 4/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.8426 - loss: 0.3838
    Epoch 5/5
    521/521 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6937 - loss: 0.7312
    ```

-----

### `batch_size=64, epochs=3, model__dim=100, model__learning_rate=0.001`

  * **Fold 1:** Total time: `36.2s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5474 - loss: 0.6885
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7391 - loss: 0.5478
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8247 - loss: 0.4019
    ```
  * **Fold 2:** Total time: `36.1s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5299 - loss: 0.6893
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7081 - loss: 0.5850
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.8102 - loss: 0.4216
    ```
  * **Fold 3:** Total time: `37.0s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.5459 - loss: 0.6809
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7607 - loss: 0.5034
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.8336 - loss: 0.3927
    ```

-----

### `batch_size=64, epochs=3, model__dim=100, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `36.2s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5323 - loss: 0.6882
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.8139 - loss: 0.4372
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8972 - loss: 0.2695
    ```
  * **Fold 2:** Total time: `36.2s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5551 - loss: 0.6766
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7337 - loss: 0.5513
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9032 - loss: 0.2457
    ```
  * **Fold 3:** Total time: `35.8s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5424 - loss: 0.6928
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.6862 - loss: 0.6168
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.8525 - loss: 0.3675
    ```

-----

### `batch_size=64, epochs=3, model__dim=128, model__learning_rate=0.001`

  * **Fold 1:** Total time: `37.2s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5457 - loss: 0.6807
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7821 - loss: 0.4678
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7100 - loss: 0.5835
    ```
  * **Fold 2:** Total time: `38.2s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5516 - loss: 0.6757
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.6713 - loss: 0.6154
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7713 - loss: 0.4822
    ```
  * **Fold 3:** Total time: `36.9s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5617 - loss: 0.6738
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.6888 - loss: 0.5895
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7548 - loss: 0.5062
    ```

-----

### `batch_size=64, epochs=3, model__dim=128, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `36.5s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.5286 - loss: 0.6898
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7681 - loss: 0.5111
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7981 - loss: 0.4621
    ```
  * **Fold 2:** Total time: `36.6s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5334 - loss: 0.6883
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7038 - loss: 0.5976
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8156 - loss: 0.4217
    ```
  * **Fold 3:** Total time: `36.7s`
    ```
    Epoch 1/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.5284 - loss: 0.6866
    Epoch 2/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7520 - loss: 0.5291
    Epoch 3/3
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8426 - loss: 0.3696
    ```

-----

### `batch_size=64, epochs=5, model__dim=100, model__learning_rate=0.001`

  * **Fold 1:** Total time: `56.9s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.5514 - loss: 0.6812
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7694 - loss: 0.4833
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8269 - loss: 0.3942
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8596 - loss: 0.3345
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8492 - loss: 0.3589
    ```
  * **Fold 2:** Total time: `57.0s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5476 - loss: 0.6749
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7226 - loss: 0.5406
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7393 - loss: 0.5341
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7686 - loss: 0.4886
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8184 - loss: 0.4026
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.5458 - loss: 0.6863
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7070 - loss: 0.5776
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8117 - loss: 0.4168
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8028 - loss: 0.4372
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7988 - loss: 0.4170
    ```

-----

### `batch_size=64, epochs=5, model__dim=100, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `56.5s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.5386 - loss: 0.6895
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7513 - loss: 0.5669
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8967 - loss: 0.2564
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9476 - loss: 0.1494
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9911 - loss: 0.0430
    ```
  * **Fold 2:** Total time: `57.4s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.5402 - loss: 0.6859
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8185 - loss: 0.4100
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9221 - loss: 0.2094
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9602 - loss: 0.1197
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9939 - loss: 0.0319
    ```
  * **Fold 3:** Total time: `1.1min`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.5576 - loss: 0.6765
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7317 - loss: 0.5501
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.8094 - loss: 0.4290
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8960 - loss: 0.2688
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9381 - loss: 0.1726
    ```

-----

### `batch_size=64, epochs=5, model__dim=128, model__learning_rate=0.001`

  * **Fold 1:** Total time: `58.6s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5338 - loss: 0.6897
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.6764 - loss: 0.6062
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.7156 - loss: 0.5620
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.7504 - loss: 0.5034
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.7965 - loss: 0.4412
    ```
  * **Fold 2:** Total time: `58.0s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5348 - loss: 0.6876
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7306 - loss: 0.5332
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8378 - loss: 0.3625
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.9373 - loss: 0.1703
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.9781 - loss: 0.0734
    ```
  * **Fold 3:** Total time: `58.5s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5382 - loss: 0.6820
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7326 - loss: 0.5443
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.7701 - loss: 0.4989
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8530 - loss: 0.3432
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8793 - loss: 0.2995
    ```

-----

### `batch_size=64, epochs=5, model__dim=128, model__learning_rate=0.0005`

  * **Fold 1:** Total time: `57.9s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5559 - loss: 0.6805
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.7438 - loss: 0.5265
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8895 - loss: 0.2765
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9597 - loss: 0.1238
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.9921 - loss: 0.0388
    ```
  * **Fold 2:** Total time: `1.1min`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.5226 - loss: 0.6885
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.7959 - loss: 0.4576
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9407 - loss: 0.1829
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9843 - loss: 0.0630
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9993 - loss: 0.0098
    ```
  * **Fold 3:** Total time: `57.2s`
    ```
    Epoch 1/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.5379 - loss: 0.6884
    Epoch 2/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7333 - loss: 0.5569
    Epoch 3/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8727 - loss: 0.3061
    Epoch 4/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8843 - loss: 0.2864
    Epoch 5/5
    261/261 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.8482 - loss: 0.3736
    ```

-----

## Final Model Training (Best Parameters)

This is the final training run, likely using the best parameters found during cross-validation on the entire training dataset.

```
Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.5545 - loss: 0.6770
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.8256 - loss: 0.3892
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.8567 - loss: 0.3368
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - accuracy: 0.8744 - loss: 0.3016
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9036 - loss: 0.2523
```

-----

## Grid Search Summary

**Best Score:** `0.8193222569315539`

**Best Parameters:**

  * `batch_size`: 64
  * `epochs`: 5
  * `model__dim`: 100
  * `model__learning_rate`: 0.0005

### Detailed Results

| Mean Score | Standard Deviation | Parameters |
| :--- | :--- | :--- |
| 0.695716 | 0.067640 | `{'batch_size': 32, 'epochs': 3, 'model__dim': 100, 'model__learning_rate': 0.001}` |
| 0.729801 | 0.082677 | `{'batch_size': 32, 'epochs': 3, 'model__dim': 100, 'model__learning_rate': 0.0005}` |
| 0.686718 | 0.037674 | `{'batch_size': 32, 'epochs': 3, 'model__dim': 128, 'model__learning_rate': 0.001}` |
| 0.733202 | 0.050825 | `{'batch_size': 32, 'epochs': 3, 'model__dim': 128, 'model__learning_rate': 0.0005}` |
| 0.668197 | 0.049250 | `{'batch_size': 32, 'epochs': 5, 'model__dim': 100, 'model__learning_rate': 0.001}` |
| 0.747004 | 0.070513 | `{'batch_size': 32, 'epochs': 5, 'model__dim': 100, 'model__learning_rate': 0.0005}` |
| 0.756280 | 0.015815 | `{'batch_size': 32, 'epochs': 5, 'model__dim': 128, 'model__learning_rate': 0.001}` |
| 0.725803 | 0.059828 | `{'batch_size': 32, 'epochs': 5, 'model__dim': 128, 'model__learning_rate': 0.0005}` |
| 0.737758 | 0.078874 | `{'batch_size': 64, 'epochs': 3, 'model__dim': 100, 'model__learning_rate': 0.001}` |
| 0.813519 | 0.029670 | `{'batch_size': 64, 'epochs': 3, 'model__dim': 100, 'model__learning_rate': 0.0005}` |
| 0.710204 | 0.068812 | `{'batch_size': 64, 'epochs': 3, 'model__dim': 128, 'model__learning_rate': 0.001}` |
| 0.804160 | 0.009153 | `{'batch_size': 64, 'epochs': 3, 'model__dim': 128, 'model__learning_rate': 0.0005}` |
| 0.764318 | 0.029990 | `{'batch_size': 64, 'epochs': 5, 'model__dim': 100, 'model__learning_rate': 0.001}` |
| **0.819322** | **0.039911** | **`{'batch_size': 64, 'epochs': 5, 'model__dim': 100, 'model__learning_rate': 0.0005}`** |
| 0.722921 | 0.015992 | `{'batch_size': 64, 'epochs': 5, 'model__dim': 128, 'model__learning_rate': 0.001}` |
| 0.812721 | 0.020715 | `{'batch_size': 64, 'epochs': 5, 'model__dim': 128, 'model__learning_rate': 0.0005}` |